# NHANES - Logistic Regression

## Load libraries

In [1]:
library(dplyr)
library(ggplot2)
library(survey)
library(knitr)
library(kableExtra)
library(IRdisplay)
library(broom)
library(sjPlot);


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: grid

Loading required package: Matrix

Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




## Load data

In [2]:
nhanes <- read.csv("data/nhanes.csv")

## Set up survey design object

In [3]:
design <- svydesign(ids = ~masked.variance.psu, 
                           strata = ~masked.variance.stratum, 
                           weights = ~MEC10YR,
                           nest = TRUE,
                           data = nhanes)

In [4]:
# View the survey design object
print(design)

Stratified 1 - level Cluster Sampling design (with replacement)
With (153) clusters.
svydesign(ids = ~masked.variance.psu, strata = ~masked.variance.stratum, 
    weights = ~MEC10YR, nest = TRUE, data = nhanes)


## Simple

### Ferritin

#### Binary Predictor

In [ ]:
ferr_model_all <- svyglm(depression ~ ID_ferr,  
                           design, 
                           family = quasibinomial())

# Print the model summary
summary(ferr_model_all)

In [ ]:
ferr_model_all <- svyglm(depression ~ ID_ferr * factor(low_income),  
                           design, 
                           family = quasibinomial())

# Print the model summary
summary(ferr_model_all)

In [ ]:
ferr_model_not_pregnant <- svyglm(depression ~ ID_ferr,  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(ferr_model_not_pregnant)

In [ ]:
ferr_model_not_pregnant <- svyglm(depression ~ ID_ferr * factor(low_income),  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(ferr_model_not_pregnant)

In [ ]:
ferr_model_preg_and_postpartum <- svyglm(depression ~ ID_ferr,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status %in% c(1, 2)))

# Print the model summary
summary(ferr_model_preg_and_postpartum)

In [ ]:
ferr_model_preg_and_postpartum <- svyglm(depression ~ ID_ferr * factor(low_income),  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status %in% c(1, 2)))

# Print the model summary
summary(ferr_model_preg_and_postpartum)


In [ ]:
ferr_model_pregnant <- svyglm(depression ~ ID_ferr * factor(low_income),  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary
summary(ferr_model_pregnant)

In [ ]:
ferr_model_postpartum <- svyglm(depression ~ ID_ferr,  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2 & low_income==1))

# Print the model summary
summary(ferr_model_postpartum)

### TfR

In [ ]:
tfr_model_all <- svyglm(depression ~ ID_tfr * factor(low_income),  
                           design, 
                           family = quasibinomial())

# Print the model summary
summary(tfr_model_all)

In [ ]:
tfr_model_not_pregnant <- svyglm(depression ~ ID_tfr * factor(low_income),  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(tfr_model_not_pregnant)

In [ ]:
tfr_model_preg_and_postpartum <- svyglm(depression ~ ID_tfr * factor(low_income),  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status %in% c(1, 2)))

# Print the model summary
summary(tfr_model_preg_and_postpartum)

In [ ]:
tfr_model_pregnant <- svyglm(depression ~ ID_tfr * factor(low_income),  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary
summary(tfr_model_pregnant)

In [ ]:
tfr_model_pregnant <- svyglm(depression ~ ID_tfr * factor(low_income),  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary
summary(tfr_model_pregnant)

## Transferrin Receptor

### N calculations

#### unweighted

In [ ]:
#Calculate the number of observations with ID_tfr == 1 and low_income == 0
# for each pregnancy.status category
nhanes %>%
  filter(ID_tfr == 1, low_income == 1) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

In [ ]:
#Calculate the number of observations with ID_tfr == 1 and low_income == 1
# for each pregnancy.status category
nhanes %>%
  filter(ID_tfr == 1, low_income == 1) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

### Continuous predictor model

#### Not pregnant

In [ ]:
# Fit the model on a subset where pregnancy.status = 0
tfr_np_model  <- svyglm(depression ~ tfr, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(tfr_np_model)

###### Model 1

In [ ]:
# Fit the model on a subset where pregnancy.status = 0
tfr_np_model_cont1  <- svyglm(depression ~ tfr + low_income + tfr:low_income, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(tfr_np_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_np_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr,
                               design = design,
                               subset = (pregnancy.status == 0 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr,
                           design = design,
                           subset = (pregnancy.status == 0 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

##### Model 2

In [ ]:
# Fit the model on a subset where pregnancy.status = 0
tfr_np_model_cont2  <- svyglm(depression ~ tfr + low_income + tfr:low_income + race.ethnicity + age + hscrp, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary
summary(tfr_np_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_np_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                               design = design,
                               subset = (pregnancy.status == 0 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                           design = design,
                           subset = (pregnancy.status == 0 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

#### Pregnant

In [ ]:
# Fit the model on a subset where pregnancy.status = 0
tfr_np_model  <- svyglm(depression ~ tfr, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary
summary(tfr_np_model)

##### Model 1

In [ ]:
# Fit the binary predictor model on a subset where pregnancy.status = 1
# Adjust for 'low_income' and include an interaction term between 'factor(ID_tfr)' and 'low_income'
tfr_preg_model_cont1 <- svyglm(depression ~ tfr + low_income + tfr:low_income, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary
summary(tfr_preg_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_preg_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr,
                               design = design,
                               subset = (pregnancy.status == 1 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr,
                           design = design,
                           subset = (pregnancy.status == 1 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

##### Model 2

In [ ]:
tfr_preg_model_cont2 <- svyglm(depression ~ tfr + low_income + tfr:low_income + race.ethnicity + age + hscrp, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary
summary(tfr_preg_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_preg_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                               design = design,
                               subset = (pregnancy.status == 1 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                           design = design,
                           subset = (pregnancy.status == 1 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

#### Postpartum

In [ ]:
# Fit the model on a subset where pregnancy.status = 0
tfr_np_model  <- svyglm(depression ~ tfr, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary
summary(tfr_np_model)

##### Model 1

In [ ]:
tfr_postpartum_model_cont1 <- svyglm(depression ~ tfr + low_income + tfr:low_income, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary
summary(tfr_postpartum_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_postpartum_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr,
                               design = design,
                               subset = (pregnancy.status == 2 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr,
                           design = design,
                           subset = (pregnancy.status == 2 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

##### Model 2

In [ ]:
tfr_postpartum_model_cont2 <- svyglm(depression ~ tfr + low_income + tfr:low_income + race.ethnicity + age + hscrp, 
                           design = design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary
summary(tfr_postpartum_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(tfr_postpartum_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                               design = design,
                               subset = (pregnancy.status == 2 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                           design = design,
                           subset = (pregnancy.status == 2 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

### Tables

In [ ]:
# Fit the models
## Not pregnant group
model_non_low_income_np <- svyglm(formula = depression ~ tfr,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 0),
                                family = quasibinomial())

model_low_income_np <- svyglm(formula = depression ~ tfr,
                            design = design,
                            subset = (pregnancy.status == 0 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_np <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 0 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_np <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 1),
                                family = quasibinomial())

## Pregnant group
model_non_low_income_p <- svyglm(formula = depression ~ tfr,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 0),
                                family = quasibinomial())

model_low_income_p <- svyglm(formula = depression ~ tfr,
                            design = design,
                            subset = (pregnancy.status == 1 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_p <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 1 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_p <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 1),
                                family = quasibinomial())

## Postpartum group
model_non_low_income_pp <- svyglm(formula = depression ~ tfr,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 0),
                                family = quasibinomial())

model_low_income_pp <- svyglm(formula = depression ~ tfr,
                            design = design,
                            subset = (pregnancy.status == 2 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_pp <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 2 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_pp <- svyglm(formula = depression ~ tfr + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 1),
                                family = quasibinomial())

In [ ]:
# Function to extract model results
extract_model_results <- function(model_summary) {
  result_table <- as.data.frame(model_summary$coefficients)
  colnames(result_table) <- c("Estimate", "Std. Error", "t value", "P-value")
  result_table$`Odds Ratio` <- exp(result_table$Estimate)
  result_table$`95% CI Lower` <- exp(result_table$Estimate - 1.96 * result_table$`Std. Error`)
  result_table$`95% CI Upper` <- exp(result_table$Estimate + 1.96 * result_table$`Std. Error`)
  result_table <- result_table[, c("Odds Ratio", "95% CI Lower", "95% CI Upper", "P-value")]
  row.names(result_table) <- make.names(row.names(result_table), unique = TRUE)
  return(result_table)
}

# Print results for all models
cat("Not pregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_np)), digits = 3)

cat("\nNot pregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_np)), digits = 3)

cat("\nNot pregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_np)), digits = 3)

cat("\nNot pregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_np)), digits = 3)

cat("\nPregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_p)), digits = 3)

cat("\nPregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_p)), digits = 3)

cat("\nPregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_p)), digits = 3)

cat("\nPregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_p)), digits = 3)

cat("\nPostpartum, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_pp)), digits = 3)

cat("\nPostpartum, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_pp)), digits = 3)

cat("\nPostpartum, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_pp)), digits = 3)

cat("\nPostpartum, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_pp)), digits = 3)

## Ferritin

### Counts

#### unweighted

In [ ]:
#Calculate the number of observations with ID_ferr == 1 and low_income == 0
# for each pregnancy.status category
nhanes %>%
  filter(ID_ferr == 1, low_income == 0) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

In [ ]:
#Calculate the number of observations with ID_ferr == 1 and low_income == 1
# for each pregnancy.status category
nhanes %>%
  filter(ID_ferr == 1, low_income == 1) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

In [ ]:
#Calculate the number of observations with ID_ferr == 0 and low_income == 0
# for each pregnancy.status category
nhanes %>%
  filter(ID_ferr == 0, low_income == 0) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

In [ ]:
#Calculate the number of observations with ID_ferr == 0 and low_income == 1
# for each pregnancy.status category
nhanes %>%
  filter(ID_ferr == 0, low_income == 1) %>%
  group_by(pregnancy.status) %>%
  summarise(count = n())

### Continuous predictor model

In [ ]:
ferr_model_all <- svyglm(depression ~ ferritin,  
                           design, 
                           family = quasibinomial())

# Print the model summary
summary(ferr_model_all)

#### Not pregnant

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 0
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_np_model <- svyglm(depression ~ ferritin, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_np_model)

##### Model 1

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 0
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_np_model_cont1 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income,  
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_np_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_np_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

##### Model 2

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 0
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_np_model_cont2 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income + race.ethnicity + age + hscrp, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 0))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_np_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_np_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

#### Pregnant

In [ ]:
ferr_preg_model <- svyglm(depression ~ ferritin, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_preg_model)

##### Model 1

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 1
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_preg_model_cont1 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_preg_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_preg_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

##### Model 2

In [ ]:
ferr_preg_model_cont2 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income  + race.ethnicity + age + hscrp, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 1))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_preg_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_preg_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

#### Postpartum

In [ ]:
ferr_postpartum_model <- svyglm(depression ~ ferritin, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_postpartum_model)

##### Model 1

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 2
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_postpartum_model_cont1 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_postpartum_model_cont1)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_postpartum_model_cont1))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

**Higher ferritin = higher risk of depression but only for low income??

the log-odds of depression for low-income and non-low-income individuals separately:

For non-low-income individuals (low_income = 0): The effect of ferritin is -0.01870.
For low-income individuals (low_income = 1): The effect of ferritin is -0.01870 + 0.02596 = 0.00726.
This indicates that for non-low-income individuals, a one-unit increase in ferritin is associated with a slight decrease in the log-odds of depression, while for low-income individuals, a one-unit increase in ferritin is associated with a slight increase in the log-odds of depression.

To interpret the results in terms of odds ratios, you can exponentiate the coefficients. For example, exp(-0.01870) = 0.981, meaning that for non-low-income individuals, a one-unit increase in ferritin is associated with a 1.9% decrease in the odds of depression (although not statistically significant).

##### Stratified

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ ferritin,
                               design = design,
                               subset = (pregnancy.status == 2 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ ferritin,
                           design = design,
                           subset = (pregnancy.status == 2 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

**Results interpretation: For non-low-income individuals, higher ferritin levels are associated with a slightly lower odds of depression**

##### Model 2

In [ ]:
# Fit the logistic regression model on a subset where pregnancy.status = 2
# Adjust for 'low_income' and include an interaction term between 'ferritin' and 'low_income'
ferr_postpartum_model_cont2 <- svyglm(depression ~ ferritin + low_income + ferritin:low_income + race.ethnicity + age + hscrp, 
                           design, 
                           family = quasibinomial(),
                           subset = (pregnancy.status == 2))

# Print the model summary to see the effects of 'ferritin', 'low_income', and their interaction on depression
summary(ferr_postpartum_model_cont2)

In [ ]:
# Extract the coefficients and standard errors
coef_table <- coef(summary(ferr_postpartum_model_cont2))

# Calculate the odds ratios
odds_ratios <- exp(coef_table[, "Estimate"])

# Calculate the 95% confidence intervals
ci_lower <- exp(coef_table[, "Estimate"] - 1.96 * coef_table[, "Std. Error"])
ci_upper <- exp(coef_table[, "Estimate"] + 1.96 * coef_table[, "Std. Error"])

# Combine the odds ratios and confidence intervals into a table
or_table <- cbind(odds_ratios, ci_lower, ci_upper)
colnames(or_table) <- c("Odds Ratio", "95% CI Lower", "95% CI Upper")

# Print the table
print(or_table)

In [ ]:
# Model for non-low-income group (low_income == 0)
model_non_low_income <- svyglm(formula = depression ~ ferritin + race.ethnicity + age + hscrp,
                               design = design,
                               subset = (pregnancy.status == 2 & low_income == 0),
                               family = quasibinomial())

# Model for low-income group (low_income == 1)
model_low_income <- svyglm(formula = depression ~ ferritin + race.ethnicity + age + hscrp,
                           design = design,
                           subset = (pregnancy.status == 2 & low_income == 1),
                           family = quasibinomial())

# Print the summary of the models
summary(model_non_low_income)
summary(model_low_income)

### Tables

#### Simple models

##### Crude

In [ ]:
ferr_model_all <- svyglm(depression ~ ID_ferr,  
                           design, 
                           family = quasibinomial())

ferr_model_np <- svyglm(depression ~ ID_ferr,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==0))


ferr_model_preg <- svyglm(depression ~ ID_ferr,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==1))

ferr_model_postpartum <- svyglm(depression ~ ID_ferr,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==2))

In [ ]:
# Function to extract model results
extract_model_results <- function(model_summary) {
  result_table <- as.data.frame(model_summary$coefficients)
  colnames(result_table) <- c("Estimate", "Std. Error", "t value", "P-value")
  result_table$`Odds Ratio` <- exp(result_table$Estimate)
  result_table$`95% CI Lower` <- exp(result_table$Estimate - 1.96 * result_table$`Std. Error`)
  result_table$`95% CI Upper` <- exp(result_table$Estimate + 1.96 * result_table$`Std. Error`)
  result_table <- result_table[, c("Odds Ratio", "95% CI Lower", "95% CI Upper", "P-value")]
  row.names(result_table) <- make.names(row.names(result_table), unique = TRUE)
  return(result_table)
}

# Print results for all models
cat("All:\n")
knitr::kable(extract_model_results(summary(ferr_model_all)), digits = 3)

cat("\nNot pregnant:\n")
knitr::kable(extract_model_results(summary(ferr_model_np)), digits = 3)

cat("\nPregnant:\n")
knitr::kable(extract_model_results(summary(ferr_model_preg)), digits = 3)

cat("\nPostpartum:\n")
knitr::kable(extract_model_results(summary(ferr_model_postpartum)), digits = 3)



##### Adjusted

In [ ]:
ferr_model_all <- svyglm(depression ~ ID_ferr + bmi + low_income + race.ethnicity + age + hscrp,  
                           design, 
                           family = quasibinomial())

ferr_model_np <- svyglm(depression ~ ID_ferr + bmi + low_income + race.ethnicity + age + hscrp,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==0))

ferr_model_preg <- svyglm(depression ~ ID_ferr + bmi + low_income + race.ethnicity + age + hscrp,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==1))

ferr_model_postpartum <- svyglm(depression ~ ID_ferr + bmi + low_income + race.ethnicity + age + hscrp,  
                                         design, 
                                         family = quasibinomial(),
                                         subset = (pregnancy.status==1))

In [ ]:
# Function to extract model results
extract_model_results <- function(model_summary) {
  result_table <- as.data.frame(model_summary$coefficients)
  colnames(result_table) <- c("Estimate", "Std. Error", "t value", "P-value")
  result_table$`Odds Ratio` <- exp(result_table$Estimate)
  result_table$`95% CI Lower` <- exp(result_table$Estimate - 1.96 * result_table$`Std. Error`)
  result_table$`95% CI Upper` <- exp(result_table$Estimate + 1.96 * result_table$`Std. Error`)
  result_table <- result_table[, c("Odds Ratio", "95% CI Lower", "95% CI Upper", "P-value")]
  row.names(result_table) <- make.names(row.names(result_table), unique = TRUE)
  return(result_table)
}

# Print results for all models
cat("All:\n")
knitr::kable(extract_model_results(summary(ferr_model_all)), digits = 3)

cat("\nNot pregnant:\n")
knitr::kable(extract_model_results(summary(ferr_model_np)), digits = 3)

cat("\nPregnant:\n")
knitr::kable(extract_model_results(summary(ferr_model_preg)), digits = 3)

cat("\nPostpartum:\n")
knitr::kable(extract_model_results(summary(ferr_model_postpartum)), digits = 3)

#### Cotinuous predictor stratified

In [17]:
# Fit the models
## all
model_non_low_income_all <- svyglm(depression ~ ferritin,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 0)

model_low_income_all <- svyglm(depression ~ ferritin,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 1)


model_non_low_income_adj_all <- svyglm(depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 0)

model_low_income_adj_all <- svyglm(depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 1)

## Not pregnant group
model_non_low_income_np <- svyglm(formula = depression ~ ferritin,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 0),
                                family = quasibinomial())

model_low_income_np <- svyglm(formula = depression ~ ferritin,
                            design = design,
                            subset = (pregnancy.status == 0 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_np <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 0 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_np <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 1),
                                family = quasibinomial())

## Pregnant group
model_non_low_income_p <- svyglm(formula = depression ~ ferritin,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 0),
                                family = quasibinomial())

model_low_income_p <- svyglm(formula = depression ~ ferritin,
                            design = design,
                            subset = (pregnancy.status == 1 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_p <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 1 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_p <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 1),
                                family = quasibinomial())

## Postpartum group
model_non_low_income_pp <- svyglm(formula = depression ~ ferritin,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 0),
                                family = quasibinomial())

model_low_income_pp <- svyglm(formula = depression ~ ferritin,
                            design = design,
                            subset = (pregnancy.status == 2 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_pp <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 2 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_pp <- svyglm(formula = depression ~ ferritin + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 1),
                                family = quasibinomial())

In [18]:
# Function to extract model results
extract_model_results <- function(model_summary) {
  result_table <- as.data.frame(model_summary$coefficients)
  colnames(result_table) <- c("Estimate", "Std. Error", "t value", "P-value")
  result_table$`Odds Ratio` <- exp(result_table$Estimate)
  result_table$`95% CI Lower` <- exp(result_table$Estimate - 1.96 * result_table$`Std. Error`)
  result_table$`95% CI Upper` <- exp(result_table$Estimate + 1.96 * result_table$`Std. Error`)
  result_table <- result_table[, c("Odds Ratio", "95% CI Lower", "95% CI Upper", "P-value")]
  row.names(result_table) <- make.names(row.names(result_table), unique = TRUE)
  return(result_table)
}

# Print results for all models
cat("All, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_all)), digits = 3)

cat("\nAll, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_all)), digits = 3)

cat("\nAll, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_all)), digits = 3)

cat("\nAll, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_all)), digits = 3)

cat("Not pregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_np)), digits = 3)

cat("\nNot pregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_np)), digits = 3)

cat("\nNot pregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_np)), digits = 3)

cat("\nNot pregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_np)), digits = 3)

cat("\nPregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_p)), digits = 3)

cat("\nPregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_p)), digits = 3)

cat("\nPregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_p)), digits = 3)

cat("\nPregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_p)), digits = 3)

cat("\nPostpartum, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_pp)), digits = 3)

cat("\nPostpartum, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_pp)), digits = 3)

cat("\nPostpartum, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_pp)), digits = 3)

cat("\nPostpartum, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_pp)), digits = 3)

All, not low-income group:




|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.068|        0.052|        0.088|   0.000|
|ferritin     |      1.002|        0.999|        1.005|   0.242|


All, low-income group:




|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.175|         0.15|        0.205|   0.000|
|ferritin     |      1.001|         1.00|        1.003|   0.122|


All, not low-income group (adjusted):


...



|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.082|        0.049|        0.138|   0.000|
|ferritin     |      1.007|        1.001|        1.013|   0.026|


Postpartum, not low-income group (adjusted):




|               | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:--------------|----------:|------------:|------------:|-------:|
|X.Intercept.   |      0.019|        0.000|       10.838|   0.230|
|ferritin       |      0.984|        0.959|        1.010|   0.226|
|bmi            |      1.055|        0.951|        1.169|   0.321|
|race.ethnicity |      0.879|        0.563|        1.371|   0.573|
|age            |      1.003|        0.884|        1.138|   0.966|
|hscrp          |      1.040|        0.950|        1.139|   0.400|


Postpartum, low-income group (adjusted):




|               | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:--------------|----------:|------------:|------------:|-------:|
|X.Intercept.   |      2.711|        0.193|       38.142|   0.464|
|ferritin       |      1.009|        1.002|        1.017|   0.023|
|bmi            |      0.987|        0.913|        1.067|   0.747|
|race.ethnicity |      1.069|        0.757|        1.509|   0.707|
|age            |      0.874|        0.787|        0.971|   0.016|
|hscrp          |      1.007|        0.924|        1.097|   0.878|

#### Binary Predictor stratified

In [13]:
# Fit the models
## all
model_non_low_income_all <- svyglm(depression ~ ID_ferr,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 0)

model_low_income_all <- svyglm(depression ~ ID_ferr,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 1)


model_non_low_income_adj_all <- svyglm(depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 0)

model_low_income_adj_all <- svyglm(depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                  design,
                                  family = quasibinomial(),
                                  subset = low_income == 1)

## Not pregnant group
model_non_low_income_np <- svyglm(formula = depression ~ ID_ferr,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 0),
                                family = quasibinomial())

model_low_income_np <- svyglm(formula = depression ~ ID_ferr,
                            design = design,
                            subset = (pregnancy.status == 0 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_np <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 0 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_np <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 0 & low_income == 1),
                                family = quasibinomial())

## Pregnant group
model_non_low_income_p <- svyglm(formula = depression ~ ID_ferr,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 0),
                                family = quasibinomial())

model_low_income_p <- svyglm(formula = depression ~ ID_ferr,
                            design = design,
                            subset = (pregnancy.status == 1 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_p <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 1 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_p <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 1 & low_income == 1),
                                family = quasibinomial())

## Postpartum group
model_non_low_income_pp <- svyglm(formula = depression ~ ID_ferr,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 0),
                                family = quasibinomial())

model_low_income_pp <- svyglm(formula = depression ~ ID_ferr,
                            design = design,
                            subset = (pregnancy.status == 2 & low_income == 1),
                            family = quasibinomial())

model_non_low_income_adj_pp <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                    design = design,
                                    subset = (pregnancy.status == 2 & low_income == 0),
                                    family = quasibinomial())

model_low_income_adj_pp <- svyglm(formula = depression ~ ID_ferr + bmi + race.ethnicity + age + hscrp,
                                design = design,
                                subset = (pregnancy.status == 2 & low_income == 1),
                                family = quasibinomial())

In [14]:
# Function to extract model results
extract_model_results <- function(model_summary) {
  result_table <- as.data.frame(model_summary$coefficients)
  colnames(result_table) <- c("Estimate", "Std. Error", "t value", "P-value")
  result_table$`Odds Ratio` <- exp(result_table$Estimate)
  result_table$`95% CI Lower` <- exp(result_table$Estimate - 1.96 * result_table$`Std. Error`)
  result_table$`95% CI Upper` <- exp(result_table$Estimate + 1.96 * result_table$`Std. Error`)
  result_table <- result_table[, c("Odds Ratio", "95% CI Lower", "95% CI Upper", "P-value")]
  row.names(result_table) <- make.names(row.names(result_table), unique = TRUE)
  return(result_table)
}

# Print results for all models
cat("All, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_all)), digits = 3)

cat("\nAll, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_all)), digits = 3)

cat("\nAll, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_all)), digits = 3)

cat("\nAll, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_all)), digits = 3)

cat("Not pregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_np)), digits = 3)

cat("\nNot pregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_np)), digits = 3)

cat("\nNot pregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_np)), digits = 3)

cat("\nNot pregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_np)), digits = 3)

cat("\nPregnant, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_p)), digits = 3)

cat("\nPregnant, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_p)), digits = 3)

cat("\nPregnant, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_p)), digits = 3)

cat("\nPregnant, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_p)), digits = 3)

cat("\nPostpartum, not low-income group:\n")
knitr::kable(extract_model_results(summary(model_non_low_income_pp)), digits = 3)

cat("\nPostpartum, low-income group:\n")
knitr::kable(extract_model_results(summary(model_low_income_pp)), digits = 3)

cat("\nPostpartum, not low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_non_low_income_adj_pp)), digits = 3)

cat("\nPostpartum, low-income group (adjusted):\n")
knitr::kable(extract_model_results(summary(model_low_income_adj_pp)), digits = 3)

All, not low-income group:




|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.075|        0.061|        0.091|   0.000|
|ID_ferr      |      1.065|        0.667|        1.698|   0.794|


All, low-income group:




|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.193|        0.168|        0.221|   0.000|
|ID_ferr      |      0.867|        0.624|        1.205|   0.399|


All, not low-income group (adjusted):


...



|             | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:------------|----------:|------------:|------------:|-------:|
|X.Intercept. |      0.121|        0.074|        0.197|   0.000|
|ID_ferr      |      0.809|        0.247|        2.652|   0.728|


Postpartum, not low-income group (adjusted):




|               | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:--------------|----------:|------------:|------------:|-------:|
|X.Intercept.   |      0.008|        0.000|        6.450|   0.167|
|ID_ferr        |      0.297|        0.027|        3.336|   0.334|
|bmi            |      1.076|        0.977|        1.185|   0.146|
|race.ethnicity |      0.855|        0.560|        1.304|   0.472|
|age            |      0.999|        0.869|        1.147|   0.985|
|hscrp          |      1.023|        0.936|        1.117|   0.622|


Postpartum, low-income group (adjusted):




|               | Odds Ratio| 95% CI Lower| 95% CI Upper| P-value|
|:--------------|----------:|------------:|------------:|-------:|
|X.Intercept.   |      2.669|        0.192|       37.077|   0.469|
|ID_ferr        |      0.818|        0.235|        2.841|   0.753|
|bmi            |      0.984|        0.905|        1.069|   0.699|
|race.ethnicity |      1.071|        0.736|        1.559|   0.720|
|age            |      0.896|        0.813|        0.987|   0.032|
|hscrp          |      1.008|        0.924|        1.098|   0.864|